In [131]:
import os
from PyPDF2 import PdfReader
import fitz
import re,json
import pandas as pd

In [25]:
redacoes = PdfReader('Redacoes1000/Versão Reduzidamil - Lucas Felpi.pdf')

In [172]:
documents = []
for file in os.listdir('Redacoes1000'):
    redacoes = fitz.open('Redacoes1000/'+file)
    version = int(re.search('[\d]',file).group()) if re.search('[\d]',file) else 0
    for pages in redacoes.pages(2, len(redacoes)):
        red = pages.get_text().replace('\xa0',' ')
        red = re.sub(r'\n(?!\n)',' ',red)
        red = re.sub(r'[ ]+',' ',red)
        red = re.sub(r'\n ','\n',red)
        fields = red.replace('”','\"').replace('“','\"').split('\"')[:-1]
        if not fields or 'Análise Prof.' in red or not "anos" in red:
            continue
        author_data = fields[0]
        text_data = fields[1:]
        text = ''
        even=True
        for chunk in text_data:
            if not even:
                text+='\"'
                text+=chunk
                text+='\"'
            else:
                text+=chunk
            even = not even
        text = text.strip()
        documents.append({
            "author_data":author_data,
            "redacao":text,
            "year":2018+(version-1),
        })

In [174]:
df = pd.DataFrame(documents)
df.loc[df['year']==2018,'tema']="Manipulação do comportamento do usuário pelo controle de dados na internet"
df.loc[df['year']==2019,'tema']="Democratização do acesso ao cinema no Brasil"
df.loc[df['year']==2020,'tema']="O estigma associado às doenças mentais na sociedade brasileira"
df.loc[df['year']==2021,'tema']="Invisibilidade e registro civil: garantia de acesso à cidadania no Brasil"

In [181]:
#still need to manually edit the Savicevic Ortega and Gabriela Traven texts, because they have different themes
with open('Redacoes1000/all_redacoes.json','w',encoding='utf-8') as file:
    for line in df.to_dict(orient='records'):
        json.dump(line,file,ensure_ascii=False)
        file.write('\n')